# Limpieza y Preprocesamiento de Datos

Este notebook realiza un proceso completo de limpieza de datos sobre el dataset de ventas. Se aplican técnicas profesionales para garantizar la calidad de los datos antes del análisis exploratorio.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Carga de Datos Originales

Cargamos el dataset raw desde el archivo CSV. Utilizamos `encoding='latin1'` porque el archivo contiene caracteres especiales en nombres de clientes y direcciones que no están en UTF-8 estándar. La clase `Path` de pathlib nos permite manejar rutas de archivos de forma multiplataforma.

In [ ]:
data_path = Path('../data/raw/sales_data_sample.csv')
df = pd.read_csv(data_path, encoding='latin1')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")

## 2. Inspección Inicial de Datos

Realizamos una exploración preliminar del dataset para entender su estructura, tipos de datos y obtener estadísticas descriptivas básicas. Esto nos ayuda a identificar posibles problemas antes de aplicar la limpieza.

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.describe()

## 3. Análisis de Valores Nulos

Identificamos valores faltantes en cada columna. Calculamos tanto el conteo absoluto como el porcentaje de valores nulos para priorizar qué columnas requieren tratamiento. Esta información es crucial para decidir si eliminar, imputar o mantener registros con datos faltantes.

In [ ]:
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing,
    'Missing_Percentage': missing_pct
}).sort_values('Missing_Count', ascending=False)

print("Missing values summary:")
print(missing_df[missing_df['Missing_Count'] > 0])

In [ ]:
plt.figure(figsize=(10, 6))
missing_cols = missing_df[missing_df['Missing_Count'] > 0]
if not missing_cols.empty:
    missing_cols['Missing_Percentage'].plot(kind='bar')
    plt.title('Missing Values by Column')
    plt.ylabel('Percentage (%)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("No missing values found in dataset")

## 4. Tratamiento de Valores Nulos

Aplicamos estrategias diferenciadas según el tipo de columna. Para campos opcionales como STATE y POSTALCODE (que pueden no existir en ciertos países), rellenamos con 'Unknown'. Para campos críticos del negocio (ORDERNUMBER, SALES, ORDERDATE, CUSTOMERNAME), eliminamos registros incompletos ya que son esenciales para el análisis.

In [ ]:
df_clean = df.copy()

# STATE and POSTALCODE can be missing for some countries
if 'STATE' in df_clean.columns:
    df_clean['STATE'].fillna('Unknown', inplace=True)

if 'POSTALCODE' in df_clean.columns:
    df_clean['POSTALCODE'].fillna('Unknown', inplace=True)

# Drop rows with critical missing values
critical_cols = ['ORDERNUMBER', 'SALES', 'ORDERDATE', 'CUSTOMERNAME']
df_clean.dropna(subset=critical_cols, inplace=True)

print(f"Records after handling missing values: {len(df_clean)}")
print(f"Records dropped: {len(df) - len(df_clean)}")

## 5. Conversión de Tipos de Datos

Convertimos columnas a sus tipos de datos apropiados. ORDERDATE se transforma a datetime para permitir operaciones temporales. Los identificadores numéricos (YEAR_ID, MONTH_ID, QTR_ID) se convierten a enteros para asegurar consistencia en cálculos y agrupaciones posteriores.

In [ ]:
df_clean['ORDERDATE'] = pd.to_datetime(df_clean['ORDERDATE'])
df_clean['YEAR_ID'] = df_clean['YEAR_ID'].astype(int)
df_clean['MONTH_ID'] = df_clean['MONTH_ID'].astype(int)
df_clean['QTR_ID'] = df_clean['QTR_ID'].astype(int)

print("Data types after conversion:")
print(df_clean.dtypes)

## 6. Detección de Valores Atípicos

Utilizamos diagramas de caja (boxplots) para visualizar la distribución de variables numéricas clave. Luego aplicamos el método IQR (Rango Intercuartílico) para identificar outliers estadísticos. Los valores fuera del rango [Q1 - 1.5*IQR, Q3 + 1.5*IQR] se consideran atípicos, aunque no necesariamente erróneos en contextos de negocio.

In [ ]:
numeric_cols = ['QUANTITYORDERED', 'PRICEEACH', 'SALES']

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for idx, col in enumerate(numeric_cols):
    axes[idx].boxplot(df_clean[col].dropna())
    axes[idx].set_title(f'{col}')
    axes[idx].set_ylabel('Value')
plt.tight_layout()
plt.show()

In [ ]:
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

for col in numeric_cols:
    outliers = detect_outliers_iqr(df_clean, col)
    print(f"{col}: {len(outliers)} outliers ({len(outliers)/len(df_clean)*100:.2f}%)")

## 7. Detección de Duplicados

Verificamos la existencia de registros completamente duplicados utilizando el método `duplicated()`. Los duplicados exactos pueden indicar errores en la carga de datos o en el sistema transaccional. Si se encuentran, se eliminan para evitar sesgos en análisis agregados.

In [ ]:
duplicates = df_clean.duplicated().sum()
print(f"Total duplicate rows: {duplicates}")

if duplicates > 0:
    df_clean.drop_duplicates(inplace=True)
    print(f"Duplicates removed. New shape: {df_clean.shape}")

## 8. Validación de Integridad de Datos

Validamos reglas de negocio básicas: las cantidades, precios y ventas deben ser positivos. Identificamos registros que violan estas restricciones y los eliminamos, ya que representan datos corruptos o inconsistencias del sistema que no tienen sentido en el contexto comercial.

In [ ]:
# Validate numeric values
invalid_qty = df_clean[df_clean['QUANTITYORDERED'] <= 0]
invalid_price = df_clean[df_clean['PRICEEACH'] <= 0]
invalid_sales = df_clean[df_clean['SALES'] <= 0]

print(f"Invalid quantities: {len(invalid_qty)}")
print(f"Invalid prices: {len(invalid_price)}")
print(f"Invalid sales: {len(invalid_sales)}")

# Remove invalid records
df_clean = df_clean[
    (df_clean['QUANTITYORDERED'] > 0) &
    (df_clean['PRICEEACH'] > 0) &
    (df_clean['SALES'] > 0)
]

print(f"\nRecords after validation: {len(df_clean)}")

## 9. Ingeniería de Características

Creamos nuevas variables derivadas de ORDERDATE para facilitar análisis temporales: año, mes, día, día de la semana y trimestre. También calculamos REVENUE_PER_UNIT (ingreso por unidad) dividiendo SALES entre QUANTITYORDERED, una métrica útil para análisis de rentabilidad por producto.

In [ ]:
df_clean['ORDER_YEAR'] = df_clean['ORDERDATE'].dt.year
df_clean['ORDER_MONTH'] = df_clean['ORDERDATE'].dt.month
df_clean['ORDER_DAY'] = df_clean['ORDERDATE'].dt.day
df_clean['ORDER_DAYOFWEEK'] = df_clean['ORDERDATE'].dt.dayofweek
df_clean['ORDER_QUARTER'] = df_clean['ORDERDATE'].dt.quarter

# Revenue per unit
df_clean['REVENUE_PER_UNIT'] = df_clean['SALES'] / df_clean['QUANTITYORDERED']

print("New features created:")
print(df_clean[['ORDER_YEAR', 'ORDER_MONTH', 'ORDER_QUARTER', 'REVENUE_PER_UNIT']].head())

## 10. Reporte Final de Calidad de Datos

Generamos un resumen comparativo entre el dataset original y el limpio. Calculamos métricas de retención de datos para evaluar cuánta información se perdió durante la limpieza. Un porcentaje de retención alto (>95%) indica que la limpieza fue quirúrgica y no agresiva.

In [ ]:
print("="*50)
print("DATA CLEANING SUMMARY")
print("="*50)
print(f"Original records: {len(df)}")
print(f"Final records: {len(df_clean)}")
print(f"Records removed: {len(df) - len(df_clean)}")
print(f"Retention rate: {len(df_clean)/len(df)*100:.2f}%")
print(f"\nFinal shape: {df_clean.shape}")
print(f"Missing values: {df_clean.isnull().sum().sum()}")

## 11. Exportación de Datos Limpios

Guardamos el dataset procesado en formato CSV con encoding UTF-8 en el directorio processed. Este archivo será la fuente de datos para todos los análisis posteriores. También reportamos el tamaño del archivo para validar que la exportación fue exitosa.

In [ ]:
output_path = Path('../data/processed/sales_data_clean.csv')
df_clean.to_csv(output_path, index=False, encoding='utf-8')
print(f"Cleaned data saved to: {output_path}")
print(f"File size: {output_path.stat().st_size / 1024:.2f} KB")